In [23]:
alert_json = {
    "rule": {
        "id": "603122",
        "description": "Suspicious logon attempt detected",
        "level": "high",
        "mitre": {
            "tactic": ["Initial Access", "Execution"],
            "technique": ["T1078", "T1059"]
        }
    },
    "agent": {
        "name": "Agent001"
    },
    "timestamp": "2023-10-01T12:34:56Z",
    "data": {
        "win": {
            "system": {
                "eventID": "4625",
                "providerName": "Microsoft-Windows-Security-Auditing"
            },
            "eventdata": {
                "logonType": "3",
                "ipAddress": "192.168.1.100",
                "failureReason": "Unknown user name or bad password"
            }
        }
    },
    "location": "Server Room 1"
}

In [24]:
import sys
import json
import requests
import configparser
import time
import csv
import threading
from requests.auth import HTTPBasicAuth

# Read configuration from config.ini
config = configparser.ConfigParser()
config.read('config.ini')

CHAT_ID = config['telegram']['CHAT_ID']
BOT_TOKEN = config['telegram']['BOT_TOKEN']
HOOK_URL = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
DOCUMENT_URL = f"https://api.telegram.org/bot{BOT_TOKEN}/sendDocument"

# Initialize cache and lock
event_cache = {}
cache_lock = threading.Lock()


def load_and_fill_template(template_path, data):
    with open(template_path, 'r') as template_file:
        template_json = json.load(template_file)
    template = template_json['template']
    fields = template_json['fields']
    
    # Populate data dynamically
    populated_data = {key: eval(value) for key, value in fields.items()}
    return template.format(**populated_data), fields

def write_events_to_csv(events, fields):
    with open('events.csv', 'w', newline='') as csvfile:
        fieldnames = list(fields.keys())
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for event in events:
            row = {key: eval(value) for key, value in fields.items()}
            writer.writerow(row)

def get_message_data(rule_id, alert_json, event_count):
    template_path = f'{rule_id}.json'
    try:
        message, fields = load_and_fill_template(template_path, alert_json)
    except FileNotFoundError:
        # Fallback to default template
        message, fields = load_and_fill_template("default.json", alert_json)
    return {
        'chat_id': CHAT_ID,
        'text': f"{message}\n\nNumber of similar events: {event_count}"
    }, fields

def collect_events(alert_json):
    global event_cache
    event_key = (alert_json['rule']['id'], alert_json['agent']['name'])
    timestamp = time.time()
    
    with cache_lock:
        if event_key not in event_cache:
            event_cache[event_key] = []
        event_cache[event_key].append({
            'timestamp': timestamp,
            'alert_level': alert_json['rule']['level'],
            'description': alert_json['rule']['description'],
            'agent': alert_json['agent']['name'],
            'rule_id': alert_json['rule']['id']
        })

def send_aggregated_alerts():
    global event_cache
    time.sleep(15)
    
    with cache_lock:
        for event_key, events in event_cache.items():
            alert_json = events[0]  # Use the first event's alert_json for the message
            msg_data, fields = get_message_data(alert_json['rule_id'], alert_json, len(events))
            write_events_to_csv(events, fields)
            
            # Send the CSV file
            files = {'document': open('events.csv', 'rb')}
            response = requests.post(DOCUMENT_URL, data={'chat_id': CHAT_ID}, files=files)
            print(response)
            
            # Send the message
            headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
            response = requests.post(HOOK_URL, headers=headers, data=json.dumps(msg_data))
            print(response)
        event_cache.clear()

# Funktion zum Ausführen des Skripts
def run_script():
    collect_events(alert_json)

# Skript dreimal ausführen
threads = []
for _ in range(3):
    t = threading.Thread(target=run_script)
    threads.append(t)
    t.start()
    time.sleep(5)  # Wartezeit zwischen den Ausführungen

# Warten, bis alle Threads abgeschlossen sind
for t in threads:
    t.join()

# Sende eine einzige Nachricht mit der Anzahl der gesammelten Events und der CSV-Datei
send_aggregated_alerts()

<Response [200]>
<Response [200]>
